In [ ]:
!pip -q install langchain
!pip -q install langchain_community
!pip -q install pypdf
!pip -q install chromadb
!pip -q install tiktoken
!pip install -U sentence-transformers
!pip -q install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import os
f = open('/content/hfapi_key.txt')
hfapi_key=f.read()
os.environ["HF_TOKEN"] = hfapi_key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hfapi_key

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    PyPDFLoader("/content/AST-1.pdf"),
    PyPDFLoader("/content/AST-2.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)
splits = text_splitter.split_documents(docs)
print(len(splits))

40


In [ ]:
import sentence_transformers

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hfEmbedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_community.vectorstores import Chroma
persist_directory = 'docs/chroma/'

vectordb = Chroma.from_documents(
    documents=splits, # splits we created earlier
    embedding=hfEmbedding,
    persist_directory=persist_directory # save the directory
)

print(vectordb._collection.count())

40


In [ ]:
question = "what does mean by production ready code?"
docs = vectordb.similarity_search(question,k=3)
print(len(docs))
print(docs[1].page_content)

3
● Ensure that code adheres to standards like PEP8  so that it's easy for others to read and 
work.  
● Scalability and performance are also important areas to consider: The production code 
needs to be ready to be deployed to infrastructure that can be scaled.In modern web 
applications, this typically means containerization for vertical or horizontal scaling .  
 
Understanding the folder structure in production environment:


In [ ]:
docs_with_mmr=vectordb.max_marginal_relevance_search(question, k=3, fetch_k=6)

In [ ]:
docs_with_mmr[0]

Document(page_content='nature.  \n \nThe following are considerations with production code:  \n \n● Testability and maintainability: Dividing code into modules that are more extensible and \neasier to maintain & test.  \n● Separating configuration from code where possible, and ensuring that functionality is \ntested and documented.  \n● May need to refactor inefficient parts of the code base and finally, reproducibility.  \n● Ensuring version control with clear processes for tracking releases and release versions,', metadata={'page': 0, 'source': '/content/AST-1.pdf'})

In [ ]:
docs = vectordb.similarity_search(question,k=5)
for doc in docs:
    print(doc.metadata)

{'page': 0, 'source': '/content/AST-1.pdf'}
{'page': 1, 'source': '/content/AST-1.pdf'}
{'page': 0, 'source': '/content/AST-1.pdf'}
{'page': 1, 'source': '/content/AST-1.pdf'}
{'page': 0, 'source': '/content/AST-1.pdf'}


In [ ]:
from langchain_community.llms import HuggingFaceEndpoint

In [ ]:
# Basic Example (no streaming)
llm = HuggingFaceEndpoint(
    endpoint_url="https://fayjubiy2xqn36z0.us-east-1.aws.endpoints.huggingface.cloud",
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    huggingfacehub_api_token=hfapi_key
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


In [ ]:
from langchain_community.llms import HuggingFaceHub
#hf = HuggingFaceHub(repo_id="gpt2", model_kwargs={"temperature": 0.01, "max_length": 5000, "return_full_text" : False}, huggingfacehub_api_token=hfapi_key)
hf = HuggingFaceHub(repo_id="gpt2", huggingfacehub_api_token=hfapi_key)

In [ ]:
query = "please list point-wise, what does production code means?"

In [ ]:
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

In [ ]:
qa_chain1 = RetrievalQA.from_chain_type(hf,
                                       retriever=vectordb.as_retriever(search_type="mmr",search_kwargs={"k": 2, "fetch_k":6} ), # "k":2, "fetch_k":3
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=False
                                       )

In [ ]:
result = qa_chain1.invoke({"query": "what is perrito?"})

In [ ]:
result["result"]

'Use the following pieces of context to answer the question at the end.\nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\nAlways say "thanks for asking!" at the end of the answer.\nonly in a traditional software system. The  ML model  and the data  are additional \nnew elements  in machine learning systems, which impact the behavior  of the ML \nsystem  \n \n \nPackage  \nA package is a collection of Python modules, and it is a useful way for us to publish related \nfunctionality so that different project applications can install our package and make use \nof the Python modules. The files that are related to packaging are pyproject.toml ,\n\ndocumentation of the software\'s behavior. They help developers understand the \ncodebase better by acting as living examples and serving as a guide for future \ndevelopment.\nQuestion: what is perrito?\nHelpful Answer: perrito is a Python module that allows you to write Python code that is used in a P

In [ ]:
from langchain.chains import RetrievalQA
qa_chain2 = RetrievalQA.from_chain_type(hf, retriever=vectordb.as_retriever())#, return_source_documents=True)
result = qa_chain2.invoke({"query": query})

In [ ]:
!pip -q install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 52.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.2 MB/s eta 0:00:00


In [ ]:
import gradio as gr

def modelHF(query):
    return qa_chain1.invoke({"query": query})

demo = gr.Interface(fn=modelHF, inputs="textbox", outputs="textbox")

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2229b7ea9eb3a46c50.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
result["result"]

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nnature.  \n \nThe following are considerations with production code:  \n \n● Testability and maintainability: Dividing code into modules that are more extensible and \neasier to maintain & test.  \n● Separating configuration from code where possible, and ensuring that functionality is \ntested and documented.  \n● May need to refactor inefficient parts of the code base and finally, reproducibility.  \n● Ensuring version control with clear processes for tracking releases and release versions,\n\nto product environment format (.py files)  \nA) Creating different modules specific to functionality: Data Manager, Training, \nPipeline, Predict  \nB) Understanding folder structure  \nC) Separating configuration files  \nD) Concept of YAML file  \n \nINTRODUCTION:  \nWhat is a  production code?  \nProduction code is designed to b

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
qa= ConversationalRetrievalChain.from_llm(hf,
                                       retriever=vectordb.as_retriever(search_type="mmr",search_kwargs={"k": 4, "fetch_k":8} ), # "k":2, "fetch_k":3
                                       memory=memory
                                       )

In [ ]:
question = "tell me something about scalability"
result = qa.invoke({"question": question})

In [ ]:
result['answer']

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nModule 3: AST -1  \n \nTITLE: Transitioning from Research to Production Environment  \nLEARNING OBJECTIVES:  \nAt the end of the experiment, you will be able to transition from the research environment to the \nproduction environment. You will understand the concept of modularization and convert the model \ndeveloped in Jupyter Notebook into different modules tailored to specific functionalities: Data \nManager, Training, Pipeline, Predict, etc.\n\n● Ensure that code adheres to standards like PEP8  so that it's easy for others to read and \nwork.  \n● Scalability and performance are also important areas to consider: The production code \nneeds to be ready to be deployed to infrastructure that can be scaled.In modern web \napplications, this typically means containerization for vertical or horizontal scaling .  \n \nUnders

In [ ]:
print